In [13]:
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc


ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mdbtools is already the newest version (1.0.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util


In [25]:
def summarize_dataframes(dfs, years):
    """
    複数のDataFrameの統計量を計算し、年ごとにまとめたDataFrameを作成する。

    Args:
        dfs: DataFrameのリスト
        years: 年のリスト (dfsと同じ長さであること)

    Returns:
        統計量をまとめたDataFrame。DataFrameのリストと年のリストの長さが異なる場合はNoneを返す。
        DataFrameが空の場合もNoneを返す。
    """
    if not dfs or not all(not df.empty for df in dfs):
        print("Error: Empty DataFrame(s) provided.")
        return None

    if len(dfs) != len(years):
        print("Error: Length of DataFrames and years lists must be the same.")
        return None

    all_numeric_cols = set()
    for df in dfs:
        numeric_cols = df.select_dtypes(include=np.number).columns
        all_numeric_cols.update(numeric_cols)

    summary = pd.DataFrame(index=years)

    for col in all_numeric_cols:
        for i, (df, year) in enumerate(zip(dfs, years)):
            if col in df.columns:  # DataFrameに列が存在するか確認
                summary.loc[year, (col, 'mean')] = df[col].mean()
                summary.loc[year, (col, 'std')] = df[col].std()
                summary.loc[year, (col, 'min')] = df[col].min()
                summary.loc[year, (col, '25%')] = df[col].quantile(0.25) # 25%を追加
                summary.loc[year, (col, '50%')] = df[col].quantile(0.50) # 50%を追加
                summary.loc[year, (col, '75%')] = df[col].quantile(0.75) # 75%を追加
                summary.loc[year, (col, 'max')] = df[col].max()       # maxを追加

            # else:
            #     # 列が存在しない場合はNaNを設定して警告を出す
            #     print(f"Warning: Column '{col}' not found in DataFrame for year {year}.")
            #     summary.loc[year, (col, 'mean')] = np.nan
            #     summary.loc[year, (col, 'std')] = np.nan
            #     summary.loc[year, (col, 'min')] = np.nan
            #     summary.loc[year, (col, '25%')] = np.nan
            #     summary.loc[year, (col, '50%')] = np.nan
            #     summary.loc[year, (col, '75%')] = np.nan
            #     summary.loc[year, (col, 'max')] = np.nan

    return summary


In [26]:

# 使用例
table_name = 'PredictorVariables'  # テーブル名を指定

file_list = [f'kjdb_prediction_cache_{year}.accdb' for year in range(2000, 2025)]
years = range(2000, 2025)

k_a_util.prepare_pd_on_notebook()
print(len(file_list))

df_statistics=[]
for y in years:
  filepath = f'/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_{y}.accdb'
  df_summary = k_a_util.access_file_statistics(filepath, table_name)
  columns_to_drop = [ 'key_horse_id', 'desc_horse_no', 'upd', 'obj_rank']
  df = df_summary.drop(columns=columns_to_drop)
  df_statistics.append(df)

25


In [27]:
# CSVファイルから読み込む場合の例 (3つのファイルを読み込む例に変更)
try:
    summary_df_from_csv = summarize_dataframes(df_statistics, years)
    print("\nCSVから読み込んだ場合の例:")
    print(summary_df_from_csv)

except FileNotFoundError as e:
    print(f"\nCSVファイルが見つかりませんでした: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except pd.errors.EmptyDataError as e:
    print(f"\nCSVファイルが空です: {e}")
    print("上記はサンプルデータを用いた実行結果です。")
except Exception as e:  # その他の例外をキャッチ
    print(f"\nCSVファイルの処理中にエラーが発生しました: {e}")
    print("上記はサンプルデータを用いた実行結果です。")

# エラーが発生するケースのテスト


CSVから読み込んだ場合の例:
      cats_odds       mean        std    min    25%    50%    75%        max  cats_feet1   cats_rnk   cats_sex  cats_clsy   cats_trk  cats_bbscr4  key_program_id   cats_jmk    cats_jb  cats_feet4  \
2000 71831.0000  8980.4098 25395.4735 0.0000 0.0000 1.6390 3.7500 71831.0000  71831.0000 71831.0000 71831.0000 71831.0000 71831.0000   71831.0000             NaN 71831.0000 71831.0000  71831.0000   
2001 77072.0000  9635.5311 27248.4483 0.0000 0.0000 1.6244 3.7500 77072.0000  77072.0000 77072.0000 77072.0000 77072.0000 77072.0000   77072.0000             NaN 77072.0000 77072.0000  77072.0000   
2002 85145.0000 10644.6440 30102.6897 0.0000 0.0000 1.5762 3.7500 85145.0000  85145.0000 85145.0000 85145.0000 85145.0000 85145.0000   64815.0000             NaN 85145.0000 85145.0000  85145.0000   
2003 92123.0000 11516.8817 32569.7903 0.0000 0.0000 1.5268 3.7500 92123.0000  92123.0000 92123.0000 92123.0000 92123.0000 92123.0000   92123.0000             NaN 92123.0000 92123.0000  92